# Spoilage analysis

In [ ]:
# Import packages

from random_data_generation import generate_data, generate_distance_mapping
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import plotly.colors as pc
import plotly.express as px
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

The generate_data() function is responsible for generating synthetic data to simulate the relationship between sensor measurements and spoilage detection in food storage. It creates a dataset with multiple sensors, experiments, and admittance values over a range of dates.

In [ ]:
# Data generation in the format of data after cleaning and checking data (nan and outliers)
df = generate_data()

In [ ]:
df.head()

The code provided is focused on comparing the admittance values (representing spoilage) between food-related data and water control data. It processes these data points, calculates differences, and generates a final output for analysis.

In [ ]:
copy_df = df.copy()

# Split data
food_df = copy_df[copy_df['experiment'].str.contains('food', case=False, na=False)].copy()
water_df = copy_df[copy_df['experiment'].str.contains('water', case=False, na=False)].copy()

# Rename the admittance column in food_df to 'admittance_food'
food_df.rename(columns={'admittance': 'admittance_food'}, inplace=True)

# Calculate the average of 'admittance_water' for each 'date' and 'sensor'
water_avg_df = water_df.groupby(['date', 'sensor'], as_index=False)['admittance'].mean()
water_avg_df.rename(columns={'admittance': 'admittance_water_avg'}, inplace=True)

# Merge the food data with the water average data
merged_df = pd.merge(
    food_df,
    water_avg_df[['date', 'sensor', 'admittance_water_avg']],
    on=['date', 'sensor'],
    how='inner'
)

# Calculate the difference between the food and avg water values
merged_df['adm_difference'] = merged_df['admittance_food'] - merged_df['admittance_water_avg']
merged_df['adm_difference_relative'] = (
    (merged_df['adm_difference']) / merged_df['admittance_water_avg']
) * 100

# Final output
final_adm_df = merged_df.copy()
final_adm_df.head()

In [ ]:
print(final_adm_df.info())

## Estimated time to spoilage

The code below handles the following:

1. It defines a threshold for spoilage detection.
2. It calculates when each sensor's corrected_admittance exceeds the threshold.
3. It visualizes the spoilage events with a line marking the time spoilage begins.

In [ ]:
THRESHOLD = 6

colors = pc.qualitative.Plotly

fig_difference = go.Figure()
fig_admittance = go.Figure()
fig_spoilage = go.Figure()

spoilage_starts = {'spoilage': [], 'sensor_name': []}

fig_difference.add_shape(
    type="line",
    x0=merged_df['date'].min(),
    y0=THRESHOLD,
    x1=merged_df['date'].max(),
    y1=THRESHOLD,
    line=dict(color="gray", dash="dash", width=0.6)
)

# Iterate over unique sensor+experiment combinations
for sensor in merged_df['sensor'].unique():
    for experiment in merged_df[merged_df['sensor'] == sensor]['experiment'].unique():
        sensor_experiment = f"{sensor}_{experiment}"
        color_index = list(merged_df['sensor'].unique()).index(sensor) % len(colors)

        temp = merged_df[(merged_df['sensor'] == sensor) & (merged_df['experiment'] == experiment)].copy()
        temp['hours'] = (temp['date'] - temp['date'].min()).dt.total_seconds() / 3600
        temp.sort_values('date', inplace=True)

        # Resample hourly and interpolate
        temp.set_index('date', inplace=True)
        temp = temp.resample('1h').mean(numeric_only=True).interpolate(method='linear')
        temp.reset_index(inplace=True)

        # Smooth difference
        temp['adm_difference'] = gaussian_filter1d(temp['adm_difference'], sigma=6)

        # Plot admittance difference
        fig_difference.add_trace(go.Scatter(
            x=temp['date'], y=temp['adm_difference'],
            mode='lines', name=f"{sensor_experiment}"
        ))

        # Plot admittance food vs water
        fig_admittance.add_trace(go.Scatter(
            x=temp['date'], y=temp['admittance_water_avg'],
            mode='lines',
            line=dict(color=colors[color_index]),
            name=f"{sensor_experiment} (water)"
        ))
        fig_admittance.add_trace(go.Scatter(
            x=temp['date'], y=temp['admittance_food'],
            mode='lines',
            line=dict(color=colors[color_index], dash='dash'),
            name=f"{sensor_experiment} (food)"
        ))

        # Spoilage detection
        first_exceeding_timestamp = temp[temp['adm_difference'] > THRESHOLD]['date'].min()
        if pd.notna(first_exceeding_timestamp):
            spoilage_starts['spoilage'].append((first_exceeding_timestamp - temp['date'].min()).total_seconds() / 3600)
            spoilage_starts['sensor_name'].append(sensor_experiment)

            fig_difference.add_shape(type="line",
                x0=first_exceeding_timestamp, y0=-200,
                x1=first_exceeding_timestamp, y1=1000,
                line=dict(color="gray", dash="dash", width=0.6)
            )
        else:
            print(f"No exceeding timestamp for {sensor_experiment}")

# Spoilage violin plot
fig_spoilage.add_trace(go.Violin(
    name='Chicken pack variation',
    y=spoilage_starts['spoilage'],
    points="all",
    box_visible=True
))
fig_spoilage.update_traces(pointpos=-0)

# Layout updates
fig_difference.update_layout(
    title='Spoilage estimation',
    yaxis_title='Absolute Difference (µS)',
    xaxis_title='Time',
    yaxis_range=[-10, 50]
)

fig_admittance.update_layout(
    title='Admittance',
    yaxis_title='Admittance (µS)',
    xaxis_title='Time',
    yaxis_range=[-10, 300]
)

fig_spoilage.update_layout(
    title='Spoilage Time Distribution',
    yaxis_title='Time to Spoilage (h)',
    height=600,
    template="plotly_white"
)

# Show plots
fig_admittance.show()
fig_difference.show()
fig_spoilage.show()

In [ ]:
# Store time to spoilage in a dataframe
spoilage_df = pd.DataFrame(spoilage_starts)
spoilage_df['experiment'] = spoilage_df['sensor_name'].str.extract(r'(food_\d+)')
spoilage_df['sensor'] = spoilage_df['sensor_name'].str.extract(r'(sensor_\d+)')
spoilage_df

#### Time to spoilage by group

This code creates a violin plot to visualize the distribution of spoilage times across different experimental groups, using Plotly's go.Figure and go.Violin methods. It highlights the variation in spoilage times (spoilage) for each experimental group.

In [ ]:
# Time to spoilage with dots dependent on experiment

unique_groups = spoilage_df['experiment'].unique()
color_map = {group: px.colors.qualitative.Set1[i % len(px.colors.qualitative.Set1)] for i, group in enumerate(unique_groups)}

# Create a figure
fig_spoilage1 = go.Figure()

# Add a violin trace for each group
for group in unique_groups:
    fig_spoilage1.add_trace(go.Violin(
        name=group, 
        y=spoilage_df[spoilage_df['experiment'] == group]['spoilage'],  
        points="all",  # Show individual points
        box_visible=True,  # Show a box plot inside the violin plot
        marker=dict(color=color_map[group])  # Assign unique color to each group
    ))

# Update layout
fig_spoilage1.update_layout(
    title='Time to Spoilage by Group',
    yaxis_title='Time to Spoilage (h)',
    height=800,
    template="plotly_white"
)

# Show the figure
fig_spoilage1.show()

#### Cluster analysis of groups by spoilage

This code performs clustering analysis on spoilage data using the KMeans algorithm, applying the Elbow Method to determine the optimal number of clusters by evaluating the Within-Cluster Sum of Squares (WCSS) and silhouette scores. It visualizes both methods and identifies the best k for clustering based on the highest silhouette score.

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


wcss = []
silhouette_scores = []


for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(spoilage_df[['spoilage']])  # Adjust based on your dataframe and features
    wcss.append(kmeans.inertia_)


for k in range(2, 11): 
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(spoilage_df[['spoilage']])
    score = silhouette_score(spoilage_df[['spoilage']], kmeans.labels_)
    silhouette_scores.append(score)

# Plotting both the Elbow Method and Silhouette Score on a single figure
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot Elbow Method
axes[0].plot(range(1, 11), wcss)
axes[0].set_title('Elbow Method for Optimal k')
axes[0].set_xlabel('Number of Clusters (k)')
axes[0].set_ylabel('WCSS (Within-Cluster Sum of Squares)')

# Plot Silhouette Scores
axes[1].plot(range(2, 11), silhouette_scores)
axes[1].set_title('Silhouette Score for Optimal k')
axes[1].set_xlabel('Number of Clusters (k)')
axes[1].set_ylabel('Silhouette Score')

# Display both plots
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

# Print the best k based on the maximum silhouette score
best_k = range(2, 11)[silhouette_scores.index(max(silhouette_scores))]
print(f'Optimal k based on silhouette score: {best_k}')


This code performs K-means clustering on the spoilage data using the previously determined optimal number of clusters (best_k). It assigns each data point a cluster label and visualizes the results using a scatter plot, where each point represents a spoilage measurement colored by its cluster assignment. 

The data is viisualised in clusters to determine whether sensors within each group cluster together.

In [ ]:
# K-means clustering

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=best_k) 
spoilage_df['spoilage_cluster'] = kmeans.fit_predict(spoilage_df[['spoilage']])

# print(spoilage_df['spoilage_cluster'])
# print(spoilage_df)

fig_clusters = go.Figure()
group_colors = spoilage_df['experiment'].astype('category').cat.codes  

fig_clusters.add_trace(go.Scatter(
    x=spoilage_df['experiment'],
    y=spoilage_df['spoilage'],
    mode='markers',
    marker=dict(
        color=spoilage_df['spoilage_cluster'],  
        colorscale='Viridis',  
        colorbar=dict(title="Spoilage Cluster"),
        size=10  
    ),
    text=spoilage_df['experiment'],  
    name='Spoilage Cluster'
))
fig_clusters.update_layout(title='Spoilage Clusters per Group', xaxis_title='Group', yaxis_title='Spoilage Cluster', xaxis_tickangle=-45, template="plotly_white", height=700)

fig_clusters.show()

Not clustered according to groups - assess whether distance plays a role

## Adjust for sensor placement (distance)

Generated random data instead of measuring

1 being the size of a box in which sensors were placed (0 would represent sensor being directly in touch with the food)

In [ ]:
sensor_distances = generate_distance_mapping(df)
sensor_distances

In [ ]:
merge_df = spoilage_df.merge(sensor_distances[['sensor', 'experiment', 'distance']], 
                             on=['sensor', 'experiment'], how='left')

# Remove water/water_control if needed
merge_df = merge_df[~merge_df['experiment'].isin(['water', 'water_control'])]

# Check the result
print(merge_df)
print(len(merge_df))
print(merge_df['distance'].value_counts())

In [ ]:
final_adm_df['date'] = pd.to_datetime(final_adm_df['date'])
final_adm_df['date']

In [ ]:
combined_df = final_adm_df.merge(merge_df, on=['experiment','sensor'],how='left')
combined_df

### Sigmoid curve to adjust corrected adm difference

In [ ]:
params = {"day_number": [], "slope": [], "intercept": []}


combined_df["day"] = combined_df["date"].dt.date 
precorr_day_df = combined_df.groupby(['day','sensor','experiment','sensor_name','distance'])['adm_difference'].mean().reset_index()


for day in precorr_day_df['day'].unique():
    fig = go.Figure()
    df_date = precorr_day_df[precorr_day_df['day']==day]

    for device in df_date['sensor'].unique():
        df_device = df_date[df_date['sensor'] == device]
        fig.add_trace(go.Scatter(x=df_device["distance"], y=df_device["adm_difference"], mode='markers', showlegend=False, marker={'color': '#1f77b4'}))

    X = df_date[['distance']].values
    y = df_date['adm_difference'].values
    model = LinearRegression().fit(X, y)
    pred = model.predict(X)
    fig.add_trace(go.Scatter(x=df_date['distance'], y=pred, mode='markers+lines', name="Linear fit", marker={'color': '#d62728'}))

    slope = model.coef_[0]
    intercept = model.intercept_
    day_number = (pd.to_datetime(day) - pd.to_datetime(precorr_day_df['day'].min())).days
    params["day_number"].append(day_number)
    params["slope"].append(slope)
    params["intercept"].append(intercept)

    fig.update_layout(title=f'{day}: Admittance vs Distance', xaxis_title='Distance', yaxis_title='Admittance',template='plotly_white')
    fig.show()

df_params = pd.DataFrame(params)

# Plot slope and intercept over time
fig_slope = go.Figure(go.Scatter(x=df_params['day_number'], y=df_params['slope'], mode='lines+markers'))
fig_slope.update_layout(title="Slope over Time", xaxis_title="Day Number", yaxis_title="Slope", template='plotly_white')
fig_slope.show()

fig_intercept = go.Figure(go.Scatter(x=df_params['day_number'], y=df_params['intercept'], mode='lines+markers'))
fig_intercept.update_layout(title="Intercept over Time", xaxis_title="Day Number", yaxis_title="Intercept", template='plotly_white')
fig_intercept.show()

In [ ]:
def sigmoid(x, L ,x0, k, b):
        y = L / (1 + np.exp(-k*(x-x0))) + b
        return y

In [ ]:
def estimate_sigmoid_initial_params(x, y):
    x = np.array(x)
    y = np.array(y)

    # Estimate b (lower asymptote) and L (range)
    b = min(y)
    L = max(y) - b

    # Flip L for decreasing trends
    if y[0] > y[-1]:
        L = -L
        b = max(y)  # Flip b to match decreasing sigmoid

    # Estimate x0: where change is fastest
    x0 = x[np.argmax(np.abs(np.gradient(y)))]

    # Estimate k: steepness (avoid zero division)
    k = 1.0 / (max(x) - min(x) + 1e-5)

    return [L, x0, k, b]

In [ ]:
from scipy.optimize import curve_fit

x_data = df_params['day_number']

# Fit slope
y_slope = df_params['slope']
initial_params_slope = estimate_sigmoid_initial_params(x_data, y_slope)

L_s, x0_s, k_s, b_s = initial_params_slope
if L_s > 0:
    bounds_slope = ([0, 0, 0, -np.inf], [np.inf, np.inf, np.inf, np.inf])
else:
    bounds_slope = ([-np.inf, 0, 0, -np.inf], [0, np.inf, np.inf, np.inf])


sigmoid_params_slope, _ = curve_fit(sigmoid, x_data, y_slope, p0=initial_params_slope, bounds=bounds_slope, maxfev=100000)


# Fit intercept
y_intercept = df_params['intercept']
initial_params_intercept = estimate_sigmoid_initial_params(x_data, y_intercept)

L_i, x0_i, k_i, b_i = initial_params_intercept
if L_i > 0:
    bounds_intercept = ([0, 0, 0, -np.inf], [np.inf, np.inf, np.inf, np.inf])
else:
    bounds_intercept = ([-np.inf, 0, 0, -np.inf], [0, np.inf, np.inf, np.inf])


sigmoid_params_intercept, _ = curve_fit(sigmoid, x_data, y_intercept, p0=initial_params_intercept, bounds=bounds_intercept, maxfev=100000)



In [ ]:
pred_slope = sigmoid(df_params['day_number'],*sigmoid_params_slope)
pred_intercept = sigmoid(df_params['day_number'],*sigmoid_params_intercept)
pred_intercept

In [ ]:
# Plot slope and intercept over time
fig_slope = go.Figure()
fig_slope.add_trace(go.Scatter(x=df_params['day_number'], y=df_params['slope'], mode='lines+markers',name="slope"))
fig_slope.add_trace(go.Scatter(x=df_params['day_number'], y=pred_slope, mode='lines+markers', marker=dict(color="red"), name="sigmoid_fitted_slope"))
fig_slope.update_layout(title="Slope over Time", xaxis_title="Day Number", yaxis_title="Slope", template='plotly_white')
fig_slope.show()

fig_intercept = go.Figure()
fig_intercept.add_trace(go.Scatter(x=df_params['day_number'], y=df_params['intercept'], mode='lines+markers', name="intercept"))
fig_intercept.add_trace(go.Scatter(x=df_params['day_number'], y=pred_intercept, mode='lines+markers',name="sigmoid_fitted_intercept"))
fig_intercept.update_layout(title="Intercept over Time", xaxis_title="Day Number", yaxis_title="Intercept", template='plotly_white')
fig_intercept.show()

### Distance correction while preserving mean of uncorrected data

In [ ]:
def lin_reg(x, slope, intercept):
    return x * slope + intercept

mean_corrected_data = pd.DataFrame()

for date in precorr_day_df['day'].unique():
    fig = go.Figure()
    df_date = precorr_day_df[precorr_day_df['day'] == date].copy()
    
    fig.add_trace(go.Scatter(x=df_date['distance'], y=df_date["adm_difference"], mode='markers', name="bearcub", marker=dict(color="red")))

    day_number = (pd.to_datetime(date) - pd.to_datetime(precorr_day_df['day'].min())).days
    slope = pred_slope[day_number]
    intercept = 0 
    mean_corrected_admittance = df_date["adm_difference"] - lin_reg(df_date["distance"], slope=slope, intercept=intercept)

    mean_shift = np.mean(df_date["adm_difference"]) - np.mean(mean_corrected_admittance)
    print(mean_shift)
    mean_corrected_admittance += mean_shift  # Shift to preserve original mean
    assert np.isclose(np.mean(df_date["adm_difference"]), np.mean(mean_corrected_admittance)), "Mean mismatch!"

    df_date["corrected_admittance"] = mean_corrected_admittance
    mean_corrected_data = pd.concat([mean_corrected_data, df_date], ignore_index=True)

    fig.add_trace(go.Scatter(x=df_date["distance"], y=mean_corrected_admittance, mode='markers', name="corrected bearcub", marker=dict(color="green")))
    fig.update_layout(title=f'{date}: Admittance vs Distance', xaxis_title='Distance (ratio)', yaxis_title='Date')
    fig.show()

#### Interpolate data for time to spoilage measurements

In [ ]:
fig = go.Figure()

# Color map based on experiment
color_map = {group: colors[i % len(colors)] for i, group in enumerate(unique_groups)}
hourly_list = []

for bc, df_bc in mean_corrected_data.groupby('sensor_name'):
    df_bc = df_bc.sort_values(by='day').copy()
    df_bc['day'] = pd.to_datetime(df_bc['day'])

    timestamps = df_bc['day'].astype(np.int64) // 10**9  # Convert to UNIX timestamps

    # Choose interpolation method based on the number of data points
    interp_kind = 'cubic' if len(df_bc) >= 4 else 'linear'

    if len(df_bc) > 1:
        interp_func = interp1d(timestamps, df_bc['corrected_admittance'], kind=interp_kind, fill_value="extrapolate")

        # Interpolate hourly data
        hourly_timestamps = np.arange(timestamps.min(), timestamps.max(), step=3600) 
        hourly_dates = pd.to_datetime(hourly_timestamps, unit='s')
        hourly_admittance = interp_func(hourly_timestamps)
        hourly_list.append(pd.DataFrame({
            "sensor_name": bc,
            "day": hourly_dates,
            "corrected_admittance": hourly_admittance,
            "sensor": df_bc['sensor'].iloc[0],
            "experiment": df_bc['experiment'].iloc[0]
        }))

        # Get the experiment value for this sensor_name group
        experiment = df_bc['experiment'].iloc[0]
        fig.add_trace(go.Scatter(
            x=hourly_dates,
            y=hourly_admittance,
            mode='lines',
            name=f'{bc} (Hourly)',
            line=dict(color=color_map[experiment], dash='dot'),
            legendgroup=experiment,
            showlegend=False 
        ))

    # Daily trace
    experiment = df_bc['experiment'].iloc[0]
    fig.add_trace(go.Scatter(
        x=df_bc['day'],
        y=df_bc['corrected_admittance'],
        mode='markers',
        name=experiment,
        marker=dict(color=color_map[experiment], size=6),
        legendgroup=experiment,
        showlegend=(experiment not in [trace.name for trace in fig.data])  # Only show legend once per experiment
    ))

mean_hourly_data_df = pd.concat(hourly_list, ignore_index=True)

fig.update_layout(
    title="Corrected Admittance Variance (Hourly Interpolation)",
    xaxis_title="Time",
    yaxis_title="Bearcub Value",
    template="plotly_white"
)
fig.update_yaxes(range=[-20, 500])
fig.show()

Function for plotting uncorrected and corrected admittance over time

In [ ]:
def plot_grouped_bc(df, bc_label, adm, date):
    fig_difference = go.Figure() 

    colors = px.colors.qualitative.Plotly
    color_map = {group: colors[i % len(colors)] for i, group in enumerate(unique_groups)}
    legend_shown = set()

    for exp in unique_groups:
        temp = df[df['experiment'] == exp].copy()

        for bc in temp[bc_label].unique():
            temp1 = temp[temp[bc_label] == bc].copy()

            show_legend = exp not in legend_shown  

            fig_difference.add_trace(go.Scatter(
                x=temp1[date], 
                y=temp1[adm],  
                mode='lines', 
                name=exp if show_legend else None,  
                line=dict(color=color_map[exp]),
                legendgroup=exp, 
                showlegend=show_legend 
            ))

            legend_shown.add(exp) 

    fig_difference.update_layout(
        title="Admittance Difference (Time to Spoilage)",
        yaxis_title=f"{adm}",
        xaxis_title="Time",
        yaxis=dict(autorange=True), 
        xaxis_range=[df[date].min(), '2025-03-11 00:00:00'],
        template="plotly_white"
    )

    fig_difference.show()

In [ ]:
plot_grouped_bc(df=mean_hourly_data_df, bc_label='sensor',adm='corrected_admittance', date='day')
plot_grouped_bc(df=merged_df, bc_label='sensor', adm='adm_difference', date='date')

### Time to spoilage in corr data

In [ ]:
fig_difference = go.Figure() 

fig_difference.add_shape(
    type="line", 
    x0=mean_hourly_data_df['day'].min(), 
    y0=THRESHOLD, 
    x1=mean_hourly_data_df['day'].max(), 
    y1=THRESHOLD, 
    line=dict(color="gray", dash="dash", width=0.6)
)

spoilage_starts = {'spoilage': [], 'bc': []} 

legend_shown = set()  # Track which experiment legends have been shown

for bc in mean_hourly_data_df['sensor_name'].unique():
    temp = mean_hourly_data_df[mean_hourly_data_df['sensor_name'] == bc].copy()

    experiment = temp['experiment'].iloc[0]
    color = color_map[experiment]
    show_legend = experiment not in legend_shown

    fig_difference.add_trace(go.Scatter(
        x=temp['day'], 
        y=temp['corrected_admittance'],  
        mode='lines', 
        name=experiment if show_legend else None, 
        line=dict(color=color),
        legendgroup=experiment,
        showlegend=show_legend
    ))

    legend_shown.add(experiment)

    first_exceeding_timestamp = temp[temp['corrected_admittance'] > THRESHOLD]['day'].min()

    if pd.notna(first_exceeding_timestamp):
        spoilage_starts['spoilage'].append((first_exceeding_timestamp - temp['day'].min()).total_seconds() / 3600)
        spoilage_starts['bc'].append(bc)

        fig_difference.add_shape(
            type="line", 
            x0=first_exceeding_timestamp, 
            y0=0, 
            x1=first_exceeding_timestamp, 
            y1=50, 
            line=dict(color="gray", dash="dash", width=0.6)
        )
    else:
        print(f'No exceeding timestamp for {bc}')

spoil_df = pd.DataFrame(spoilage_starts)

fig_difference.update_layout(
    title="Admittance Difference (Time to spoilage)",
    yaxis_title="Corrected Admittance",
    xaxis_title="Time",
    yaxis_range=[0, 40],
    xaxis_range=['2025-03-06 12:00:00', '2025-03-09 00:00:00'],
    template="plotly_white"
)

fig_difference.show()

#### Violin plots of time to spoilage in uncorr/corr data

This code creates a plot showing the corrected admittance values over time for different sensors in relation to a defined spoilage threshold. It tracks when each sensor's corrected admittance exceeds the threshold, which signifies the start of spoilage. A vertical dashed line is drawn to indicate the timestamp when spoilage starts for each sensor. 

In [ ]:
fig_spoilage = go.Figure()
fig_spoilage.add_trace(go.Violin(name='Chicken pack variation', y=spoilage_df['spoilage'], points="all", box_visible=True))
fig_spoilage.add_trace(go.Violin(name='Chicken pack variation (corrected)', y=spoil_df['spoilage'], points="all", box_visible=True))

fig_spoilage.update_layout(title="Time to spoilage (Uncorrected vs Corrected)", yaxis_title="Hours to spoilage", template="plotly_white", height=600)

fig_spoilage.show()

Final quantification (how does the time to spoilage vary in uncorr/corr data)

In [ ]:
uncorrected_variation = spoilage_df['spoilage'].max() - spoilage_df['spoilage'].min()
corr_var = spoil_df['spoilage'].max() - spoil_df['spoilage'].min()

print(f'Uncorrected time to spoilage: {uncorrected_variation}')
print(f'Corrected time to spoilage: {corr_var}')